<a href="https://colab.research.google.com/github/ericbonelli/Cientista-de-Dados_EBAC/blob/main/Mod13_Tarefa02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [27]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import r2_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_renda = pd.read_csv('/content/drive/MyDrive/Especialização em IA/Cientista de Dados - Ebac/Desenvolvimento de aplicações com Python/Regressão 1/previsao_de_renda.csv')
df_renda.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [4]:
#Verificar dados faltantes e nulos
df_renda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [5]:
# eliminar as colunas com dados faltantes
df_renda.dropna(inplace=True)
df_renda.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             12427 non-null  int64  
 1   data_ref               12427 non-null  object 
 2   id_cliente             12427 non-null  int64  
 3   sexo                   12427 non-null  object 
 4   posse_de_veiculo       12427 non-null  bool   
 5   posse_de_imovel        12427 non-null  bool   
 6   qtd_filhos             12427 non-null  int64  
 7   tipo_renda             12427 non-null  object 
 8   educacao               12427 non-null  object 
 9   estado_civil           12427 non-null  object 
 10  tipo_residencia        12427 non-null  object 
 11  idade                  12427 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  12427 non-null  float64
 14  renda                  12427 non-null  float64
dtypes: bool

In [6]:
#Verificar dados duplicados
df_renda.duplicated().sum()
print(f'A base possui {df_renda.duplicated().sum()} dados duplicados')

A base possui 0 dados duplicados


In [7]:
# Transformar as variáveis em Dummmy
df_cleaned = df_renda.drop(columns=["Unnamed: 0", "data_ref", "id_cliente"])
df_cleaned = pd.get_dummies(df_cleaned, drop_first=True)
df_cleaned = df_cleaned.astype(int)
print(df_cleaned.info())
df_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   posse_de_veiculo               12427 non-null  int64
 1   posse_de_imovel                12427 non-null  int64
 2   qtd_filhos                     12427 non-null  int64
 3   idade                          12427 non-null  int64
 4   tempo_emprego                  12427 non-null  int64
 5   qt_pessoas_residencia          12427 non-null  int64
 6   renda                          12427 non-null  int64
 7   sexo_M                         12427 non-null  int64
 8   tipo_renda_Bolsista            12427 non-null  int64
 9   tipo_renda_Empresário          12427 non-null  int64
 10  tipo_renda_Pensionista         12427 non-null  int64
 11  tipo_renda_Servidor público    12427 non-null  int64
 12  educacao_Pós graduação         12427 non-null  int64
 13  educacao_Secundário  

,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda,sexo_M,tipo_renda_Bolsista,tipo_renda_Empresário,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,0,1,0,26,6,1,8060,0,0,1,...,0,0,1,0,0,1,0,0,0,0
1,1,1,0,28,7,2,1852,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,1,0,35,0,2,2253,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,0,1,1,30,4,3,6600,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,0,0,33,4,1,6475,1,0,0,...,0,0,1,0,0,0,0,0,0,1


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [8]:
# 1- Separe a base em treinamento e teste (25% para teste, 75% para treinamento)
from sklearn.model_selection import train_test_split
X = df_cleaned.drop('renda', axis=1)
y = df_cleaned['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((9320, 24), (3107, 24), (9320,), (3107,))

In [9]:
# Padronização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
# 2- Rode uma regularização ridge com alpha
alphas_ridge = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge_results = {}

for alpha in alphas_ridge:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train)
    y_pred = ridge.predict(X_test_scaled)
    ridge_results[alpha] = r2_score(y_test, y_pred)

print("Resultados Ridge Regression:")
for alpha, r2 in ridge_results.items():
    print(f"Alpha {alpha}: R² = {r2:.4f}")
best_ridge_alpha = max(ridge_results, key=ridge_results.get)
print(f"\nMelhor Alpha para Ridge: {best_ridge_alpha}")


Resultados Ridge Regression:
Alpha 0: R² = 0.2965
Alpha 0.001: R² = 0.2965
Alpha 0.005: R² = 0.2965
Alpha 0.01: R² = 0.2965
Alpha 0.05: R² = 0.2965
Alpha 0.1: R² = 0.2965

Melhor Alpha para Ridge: 0.1


In [12]:
# 3- Regressão LASSO. Qual método chega a um melhor resultado?
alphas_lasso = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
lasso_results = {}

for alpha in alphas_lasso:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_scaled, y_train)
    y_pred = lasso.predict(X_test_scaled)
    lasso_results[alpha] = r2_score(y_test, y_pred)

print("\nResultados LASSO Regression:")
for alpha, r2 in lasso_results.items():
    print(f"Alpha {alpha}: R² = {r2:.4f}")
best_lasso_alpha = max(lasso_results, key=lasso_results.get)
print(f"\nMelhor Alpha para LASSO: {best_lasso_alpha}")


Resultados LASSO Regression:
Alpha 0: R² = 0.2965
Alpha 0.001: R² = 0.2965
Alpha 0.005: R² = 0.2965
Alpha 0.01: R² = 0.2965
Alpha 0.05: R² = 0.2965
Alpha 0.1: R² = 0.2965

Melhor Alpha para LASSO: 0.1


In [13]:
# 4 - Rode um modelo stepwise
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# --- Creating Dummy Variables ---
X = df_cleaned.drop(columns=['renda'])
y = df_cleaned['renda']


def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.05,
                       threshold_out=0.05,
                       verbose=True):
    """ Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            # Use .loc to ensure correct data selection and type consistency
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X.loc[:, included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin() #Use idxmin() to get the name of column with the lowest p-value
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        # Use .loc to ensure correct data selection and type consistency
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X.loc[:, included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()  # null if pvalues is empty
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax() #Use idxmax() to get the name of column with the highest p-value
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 0.0
#############
['tempo_emprego']
Add  sexo_M                         with p-value 0.0
#############
['tempo_emprego', 'sexo_M']
Add  tipo_renda_Empresário          with p-value 1.76295e-07
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário']
Add  idade                          with p-value 1.1266e-07
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade']
Add  educacao_Superior completo     with p-value 4.0994e-06
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo']
Add  qt_pessoas_residencia          with p-value 0.00646758
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'qt_pessoas_residencia']
Add  posse_de_imovel                with p-value 0.0135011
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'qt_pessoas_residencia', 'posse_de

In [14]:
reg_stepwise = sm.OLS(y, sm.add_constant(pd.DataFrame(X[variaveis]))).fit()
reg_stepwise.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.254
Method:                 Least Squares   F-statistic:                     605.6
Date:                Mon, 03 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:13:19   Log-Likelihood:            -1.2880e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12419   BIC:                         2.577e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                      -3208.6120    423.510     -7.576      0.000   -4038.758   -2378.467
tempo_emprego                551.6113     10.926     50.485      0.000     530.194     573.028
sexo_M                      6084.2251    146.245     41.603      0.000    5797.563    6370.887
tipo_renda_Empresário        802.5691    154.756      5.186      0.000     499.224    1105.915
idade                         47.9495      8.096      5.922      0.000      32.080      63.819
educacao_Superior completo   649.4107    143.132      4.537      0.000     368.849     929.973
qt_pessoas_residencia        201.2694     76.178      2.642      0.008      51.949     350.590
posse_de_imovel              359.6690    145.578      2.471      0.014      74.313     645.025
==============================================================================
Omnibus:                    17574.071   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9898962.642
Skew:                           8.129   Prob(JB):                         0.00
Kurtosis:                     140.307   Cond. No.                         265.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
# 5. Comparação de Parâmetros
print("\nComparação de Modelos:")
print(f"Ridge (alpha={best_ridge_alpha}): R² = {ridge_results[best_ridge_alpha]:.4f}")
print(f"LASSO (alpha={best_lasso_alpha}): R² = {lasso_results[best_lasso_alpha]:.4f}")
print(f"Stepwise: R² = {reg_stepwise.rsquared:.4f}")


Comparação de Modelos:
Ridge (alpha=0.1): R² = 0.2965
LASSO (alpha=0.1): R² = 0.2965
Stepwise: R² = 0.2545


In [38]:
# Ridge com features polinomiais
ridge_poly = Ridge(alpha=best_ridge_alpha)
ridge_poly.fit(X_train_poly, y_train)
y_pred_poly = ridge_poly.predict(X_test_poly)
poly_r2 = r2_score(y_test, y_pred_poly)

In [37]:
# Lasso com features polinomiais
lasso_poly = Lasso(alpha=best_lasso_alpha)
lasso_poly.fit(X_train_poly, y_train)
y_pred_poly_lasso = lasso_poly.predict(X_test_poly)
poly_lasso_r2 = r2_score(y_test, y_pred_poly_lasso)

In [36]:
# stepwise com features polinomiais
reg_stepwise_poly = sm.OLS(y_train, sm.add_constant(X_train_poly)).fit()
poly_stepwise_r2 = reg_stepwise_poly.rsquared

In [45]:
# 5. Comparação de Parâmetros com as melhorias feitas
print("\nComparação de Modelos:")
print(f"\R² Ridge com Features Polinomiais: {poly_r2:.4f}")
print(f"\R² Lasso com Features Polinomiais: {poly_lasso_r2:.4f}")
print(f"\R² Stepwise com Features Polinomiais: {poly_stepwise_r2:.4f}")


Comparação de Modelos:
\R² Ridge com Features Polinomiais: 0.3871
\R² Lasso com Features Polinomiais: 0.3879
\R² Stepwise com Features Polinomiais: 0.4227


In [40]:
# 7. Árvore de Regressão
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train_scaled, y_train)
y_pred_tree = tree_reg.predict(X_test_scaled)
tree_r2 = r2_score(y_test, y_pred_tree)
print(f"\nR² Árvore de Regressão: {tree_r2:.4f}")


R² Árvore de Regressão: 0.1264


In [41]:
# Grid Search para ajuste de hiperparâmetros
param_grid = {
    'max_depth': [None, 3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring='r2'
)
grid_search.fit(X_train_scaled, y_train)


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42),
             param_grid={'max_depth': [None, 3, 5, 7, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='r2')

In [42]:
# Melhores parâmetros e predição
best_tree = grid_search.best_estimator_
y_pred_best_tree = best_tree.predict(X_test_scaled)
best_tree_r2 = r2_score(y_test, y_pred_best_tree)
print("\nMelhores Parâmetros da Árvore:")
print(grid_search.best_params_)
print(f"R² Árvore com Grid Search: {best_tree_r2:.4f}")


Melhores Parâmetros da Árvore:
{'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}
R² Árvore com Grid Search: 0.2508


In [43]:
# Random Forest (geralmente mais robusto que árvore única)
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg.fit(X_train_scaled, y_train)
y_pred_rf = rf_reg.predict(X_test_scaled)
rf_r2 = r2_score(y_test, y_pred_rf)
print(f"\nR² Random Forest: {rf_r2:.4f}")


R² Random Forest: 0.3657


In [44]:
# Grid Search para Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_grid_search = GridSearchCV(
    RandomForestRegressor(random_state=42),
    rf_param_grid,
    cv=5,
    scoring='r2'
)
rf_grid_search.fit(X_train_scaled, y_train)


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [None, 3, 5, 7],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='r2')

In [46]:
# Melhores parâmetros e predição do Random Forest
best_rf = rf_grid_search.best_estimator_
y_pred_best_rf = best_rf.predict(X_test_scaled)
best_rf_r2 = r2_score(y_test, y_pred_best_rf)
print("\nMelhores Parâmetros do Random Forest:")
print(rf_grid_search.best_params_)
print(f"R² Random Forest com Grid Search: {best_rf_r2:.4f}")


Melhores Parâmetros do Random Forest:
{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
R² Random Forest com Grid Search: 0.3811


In [47]:
# Comparação detalhada
print("\nComparação Detalhada:")
print(f"Árvore Simples R²: {tree_r2:.4f}")
print(f"Árvore com Grid Search R²: {best_tree_r2:.4f}")
print(f"Random Forest R²: {rf_r2:.4f}")
print(f"Random Forest com Grid Search R²: {best_rf_r2:.4f}")


Comparação Detalhada:
Árvore Simples R²: 0.1264
Árvore com Grid Search R²: 0.2508
Random Forest R²: 0.3657
Random Forest com Grid Search R²: 0.3811
